# 聊一聊mmaction2代码主体框架

## 一.为何要写这篇文章

最近在搞视频理解，而目前对于视频理解的官方教程并没有，主要是因为视频理解也是近几年兴起的，但是目前对于视频理解的研究流程进度也呈现出比较合理可观的状态，所以就借着自己整理代码的时候来捋一下视频理解的原理。



## 二.视频理解发展

### 1.发展概述

视频理解是一个非常大的方向，类似于基础的图像识别任务,包含了分类/分割/检测三大领域的任务。图像识别任务事实上可以精简为：
$$
backbone+other feature
$$
这样的一个结构。因为对于图像识别任务就是要先提取其中的“**主要特征**”，至于如何使用特征以及如何在后续操作中处理特征就是后续要研究的问题了。因此，较为核心的问题依然是：如何通过backbone提取主干特征。

最初，人们在视频理解领域的工作主要是围绕着“光流法”来进行，这一时期是以“**手工提取特征**”方法为主流的时期。

后续，人们才开始向视频理解引入神经网络，也就是引入深度学习方法。当时随着Alexnet、Vggnet等CNN网络的相继出现并不断应用在图像/音频等领域，“神经网络在AI领域确实是一个非常有效的工具”这个说法逐渐被人们接受。但是到了视频领域，刚开始以DeepVideo为代表的一系列视频理解深度学习任务都以失败告终——他们利用传统神经网络提取特征对视频分类的效果甚至还不如手工提取特征好。出现这一现象的主要原因是：当时学者们要么是将视频分帧后堆叠放入神经网络训练，要么就直接一张一张地放入进行训练。如此一来，神经网络都只能学习到空间上的特征到类别特征向量的映射，而不能学习到时间上的特征到类别特征向量上的映射。举个例子：把2D的图片堆叠起来放入三维卷积神经网络和把一张张2D图片逐帧放入二维卷积神经网络是一样的效果。

因此，如何让神经网络同时学习到空间/时间上的特征是一大难题。双流神经网络(Two Stream Network)开启了深度学习在视频理解领域的新时代，后续还出现了更多基于三维卷积、基于光流的方法乃至2D、3D混合的深度学习方法。


### 2.传统的光流法

#### (1)光流是什么？

光流就是计算机视觉领域中描述物体状态变化（运动）的一种物理量。简单来说，在前后连续的两张帧图片中，像素点A从1点移动到了2点，那么我就可以用一个**二维向量**来描述这个点的运动。

<img src='images/372049614f8671465a479aa67039abba64401fdf7f1f0a71e462cdd7a1372ee3.png' height=300 width=600>


计算光流的方法有很多，这里我们介绍一种：

<img src='images/2de154470d739d5ec85229616d0133046d84f4d1cb8ac2f0c4b090500c73c238.png' width=400 height=400>


对于上图假设I(x,y,z)是一个由空间x/y坐标以及时刻决定的函数，函数值代表的是某一点、某一个时刻的像素值，那么我们有如下假设：
$$
I(x,y,z)=I(x+\Delta x,y+\Delta y,z+\Delta z)(色彩/亮度恒定方程)
$$

经过一阶泰勒展开，并且化简就可以得到：
$$
\frac{\delta I}{\delta x}\Delta x+\frac{\delta I}{\delta y}\Delta y+\frac{\delta I}{\delta t}\Delta t=0
$$

两边同时除以\delta t,进一步化简得到：
$$
\frac{\delta I}{\delta x}V_x+\frac{\delta I}{\delta y}V_y=-\frac{\delta I}{\delta t}
$$

发现有一个方程，但是有两个未知数，是一个**欠定问题**，又称孔径问题——需要额外约束求解。

#### (2)光流的表达

<img src='images/d4c1428f17a1ae125bac5bdb28c9167f53ea7b4f1fd3cbf72467af2141fb0e35.png' height=350 width=600>

当然，我们也可以根据感兴趣与否，将光流表达分为稠密表达和稀疏表达。稠密表达就是所有的光流变化都表示出来，另外一个就是将我们感兴趣的部分的光流变化表示出来。

#### (3)利用光流进行视频理解

##### 1'.稠密轨迹

<img src='images/250e08e7e900c45d664676f4300d0c3c25d1ada0a0ccb1ccb8df00c672d1e7d5.png' width=600 height=400>

这种方法是一种手工设计的视频特征，先通过光流来估计每一帧、每一个点的光流轨迹，然后将光流分布表达出来。

<img src='images/6e04374b5702eeeb90cc8de8c359fba3a7bb6eda87f2537fd7caaea2b62dd758.png' width=600 height=400>


首先就是选取兴趣点，对于每一个兴趣点：

<img src='images/fa47f3c927fc9ad9e64376617cba5e456f11c908780c85c930a286cbf9470067.png' width=800 height=300>

就是“追踪”一个短时间内的光流轨迹。然后：

<img src='images/d110d4d18bf7b6273443407a67b52d19a1ce769adb9828251eba7227865dd585.png' width=800 height=250>

##### 2'.按轨迹对齐的局部特征

<img src='images/53efe89aeeb0cf6503e5a6aada578100a0387405100771bc55a225f6a4a3e58f.png' width=700 height=400>


<img src='images/55c88c95d9d45a5bc84b12211607af5400ae7fd4dd14e3e3fd47243174b5a8d7.png' width=700 height=400>

<img src='images/233824c5fee2b1d8b9a68ff18eaf58c3672c5b58ca7d9ec480177d9128e9c35a.png' width=200 height=100>

##### 3'.特征编码


<img src='images/253ab64bacca4b9c457f994a255fdb28605555ca05bef62b71607c31aa1f169e.png' width=700 height=400>

通过以上手工特征提取方法提取到的视频特征是非常多的，那么如何对视频特征进行整理和精简呢？于是就采用了如上图的特征编码方法。


##### 4'.iDT 2013

<img src='images/93e8d91acea434b8735336564d54679601d1807ce4b95611288c4d0ab7d4b4b2.png' width=700 height=400>


------

### 3.DeepVideo 2014

<img src='images/40bee4b46f0026838d01d70043818d2b39580290192fac2b6ad4e4c0bfeefc85.png' width=700 height=400>

### 3.深度学习在视频理解领域的开山之作——双流神经网络(2014年提出)

关于双流神经网络更加详细的介绍可以看我的另外一篇文章。

#### (1)双流神经网络的引入

对于空间特征(spatial feature)，我们已经可以通过卷积操作对视频帧图片的空间信息进行提取，但是对于时空特征(temporal feature)呢？当时的学者就从“手工提取特征”传统方法的“**光流法**”中获取到了灵感(不了解光流的小伙伴可以暂且把光路理解为一种能够体现**视频中帧与帧之间变化的特征**，也就是时间上的变化特征)——利用卷积操作去提取光流图（一种体现光的流动也就是时间上物体动作变化特征的图片）中的特征，从而获得光流特征，这便是深度学习在视频理解领域的开山之作——Two stream network(aka 双流神经网络)。

<img src='images/c8d0b0fa36794136e0d6b64f1adba999a4a85972d53a03a35eb08a74bd4fc963.png' width=700 height=300>

#### (2)双流神经网络的灵感来源

该网络的作者是从人体视觉中枢的工作机制中获取到了灵感：人体视觉中枢在处理动态特征的时候，一种细胞对于图像中的色彩分布非常敏感，但是对于图像的动态变化不敏感，而另外一种细胞则刚好相反，两者相辅相成才让人类高效率地处理动态的视觉信息。

#### (3)算法简单概述

这个网络的原理有两种输入：一种是图像张量，另外一种就是4-10张图片的光流图。为什么要用多张图片的光流图呢？因为光流体现的是动作的变化，那么就得用到多张图片（一张图片是不可能有“变化”），同时一般我们是按照帧进行划分视频的，那么一个动作变化对于人眼来说就是一瞬间，但是对于计算机而言可能就是4-10张图片才能判断出来。

得到了这两种输入后，空间的单帧图片输入我们的空间支流神经网络，时空的多帧图片形成的光流图输入时间流支流神经网络，最后可以分别得到两种概率值，最后输出的时候进行融合加权相乘再相加就可以的到最后的输出。

训练的时候：对于每一张图片随机选取一个时刻，前向传播对应的光流和图像。

预测的时候：会对视频的所有帧图片进行预测然后平均所有分类概率。

#### (4)取得的成就
![图 17](images/df008fcc55ec659adb75fcdcba986c5c0f36382bd75957daae1614cc1232eb1f.png)  


#### (5)问题分析

双流网络仅仅是短时建模，动作又一个单一时刻的图像和他附近的图像所形成的光流所确定，简单来说**看的太短浅**了。就比如说：刚开始视频可能是一个运动员在跑步，后来运动员突然起跳，起跳就一瞬间的事情，但是大部分是在助跑，最后机器预测出来可能就是该视频是跑步，而实际上是跳远。

因此，对于双流神经网络而言，最好的解决方法就是进行长时间的建模。

### 4.时序分段网络(2016)

<img src='images/099405ecc41a3a487b3f1b901dbaa12fd549fd620fb506e1f7ee6ef4ae308b4a.png' width=700 height=300>

#### (1)实训分段网络的思想

时序分段网络的思想就是将我们的动作建模为多个时段的图像和光流的函数。为何不把每一秒的帧图片提取出来然后分析再融合呢？原因很简单，算力不足，因此，该方法就是将我们的视频平均分为了3段，均衡了算力成本和精度。

#### (2)分段公式函数

<img src='images/5c9cddba5bd72cd05df03c0a9b7c7aab9ade9bb8dd28ec113afa5fc62725ca16.png' width=700 heigt=200>

这里提出的一个叫做分段共识函数的概念，这个函数的作用就是起到一个“融合”结果的作用。

 #### (3)TSN的其他工作

<img src='images/a07a7f1270ec8bf9399c4c280b571ddf03fc6ff8547c3902822e4ada6a4c1938.png' width=600 height=400>

#### (4)TSN取得的成就

<img src='images/48cc062be79f29e735cbc5a0c7190c0edba059582d0e1a7949d5d10381d7ad4a.png' width=500px height=300px>

<img src='images/2778f840ed55c8baa641252f9929cee0471aaea46f74b553a6305dca54a29c18.png' width=500 height=200>

-----

接下来又是一个比较新的阶段——3D卷积：

<img src='images/67a519a9916d63bbc020115d185230c11bcea4a946b9fa952d11306c38bd9a0f.png' width=500 height=300>

3D卷积就是直接让神经网络直接把这个帧图片到光流图的“映射”给学习到。

<img src='images/57dc66090885c7f6e6f685c49582c2d3233aafcf0765e099aefa2fb5fe51e9b4.png' width=500 height=300>

### 5.C3D 2014

<img src='images/66cd0528409b561a12d98a3753736bf872b430de0c0c902b315fcc59439ec855.png' width=600 height=300>

#### (1)结构特点

C3D在当时其实相较于普通的图像分类网络alexnet，基本上仅仅是将2D卷积结构替换为了3D卷积。

#### (2)C3D网络的缺陷、原因分

一般情况下，C3D的性能其实不如双流神经网络，需要额外配合一些技术才能够超过双流网络。至于原因，非常简单：
- 三维卷积的参数量更大，更难训练；
- 是自主设计的结构、从头训练，不能够直接使用预训练好的二维神经网络权重。

#### (3)成就

<img src='images/9ba00e2f643dcb3a49ff4b04517412e6faf77203587b5823b7e80eeb821a9f05.png' width=600px height=300px>


### 6.I3D——膨胀的3D神经网络 2017

I3D也是一种3D神经网络，但是它是由图像分类的二维网络“膨胀”而来，从而就可以使用已有的图像分类模型。

<img src='images/e526d99651baf490a48176f2adb84d5c49bbd16846137c169bbfe6d895475fe0.png' width=600 height=300>



#### (1)非常重要的一个概念——卷积核的膨胀
  
<img src='images/b00795b3dc0b6a16557b8198ba4c78120f964251cdd64b8e2a3cbe4552d77267.png' width=600 height=300>

所谓的膨胀，就是将原有的多个“特征图 \* 卷积核 = 输出特征图”的操作给堆叠起来，这样的话就可以称为一个三个维度上的卷积。

<img src='images/562964523b38a27405b623e5c2502940ebc33fb92b83d9f1f5d81b4705281eee.png' width=600 height=300>

#### (2)优缺点分析

优点：使用了膨胀的概念，使得我们可以使用2D的图像分类的预训练权重了；

缺点：由于膨胀本质上还是增加了参数量，训练难度依然不小。

-----

### 7.更加高效的3D卷积神经网络

混合2D和3D卷积：

![图 1](images/ab6162928bb6899551b5fec98a92e40dac60af479331632b86217aac221b2ac3.png)  


## 四.学习和分析mmaction2的主体框架

-----
学习资料:
- [B站视频](https://www.bilibili.com/video/BV13y4y1u7ro?p=2&spm_id_from=pageDriver)